## Classiq for Qiskit Users

Welcome to the "Classiq for Qiskit Users" tutorial. This guide is designed for users already familiar with Qiskit and are looking to seamlessly transition to Classiq to build quantum algorithms with a higher-level functional design that enables developers to focus on the quantum program rather than a quantum circuit using the four pillars: Design, Optimize, Analyze, Execute.

<details markdown>
<summary markdown> New to Classiq? </summary>
If you're new to Classiq, we highly recommend you to check out our <a href="https://docs.classiq.io/latest/classiq_101/">Classiq 101</a> guide to get started!

#### Table of Contents

1. [Key Differences between Qiskit and Classiq](#section1)
    1.1 Hello World :bell state preparation
    1.2. similarities:
     how we for eg apply gate
    qc.operation(qubit#) qiskit
    classqiq how we apply H gate

2. [Building Blocks of Quantum Algorithms (with Simple Exercises)](#section2)

    2.1 Preparing the initial state Q vs C
        2.1.1 Exercise: Prepare a minus state 
    2.2 Constructing an oracle Q vs C
        2.2.1 Exercise: Construct an oracle such that $f(x)$
    2.3 Transpilation
    2.4 Changing computational basis
    2.5 Exporting quantum programs
    2.6 Unique Advantage of Classiq: Optimization
        2.6.1 Example: Optimizing for Depth

3. [Quantum Algorithms](#section3)
    3.1 Phase Kickback 
        3.1.1 Phase Kickback in Qiskit
        3.1.2 Phase Kickback in Classiq

4. [Appendix Notes](#section4) QEC and Mid measurement are not enabled using Classiq

<a id="section1"></a> 1. Key Differences between Qiskit and Classiq

##### 1.1 Key Differences 
Qiskit, as we know is designed to be a python SDK for quantum programming, which start with qubits and functions which aggregate these qubits with logical quantum gates to build quantum circuits. Classiq on the other hand offers an advantage of designing quantum algorithms without being tied to circuits. How? By using a higher-level functional design. In Classiq, quantum algorithms can be built using functions which contain native primitives and datatypes like QNum and QArray, which all then culminate in a main function. In addition to its python SDK, Classiq offers a native language, QMod which is how our quantum algorithm written in python connects with the easy-to-use visual Classiq platform for synthesis, optimization, analysis, and execution. This in turn also offers quantum experts flexible design and deep optimization of quantum programs for specific implementations.

##### 1.1.2 Hello World

Welcome to the Classiq World!

##### 1.1.3 Visualization

One of the most useful visualization tools used in quantum computing is histogram to see observe and analyse the measurement results. While Qiskit has a separate sub-package <code>qiskit.visualization</code>, part of their SDK, Classiq has a seamless Jobs page which is automatically show after the user executes their quantum program. Let's explore how we can arrive at a measurement result in the histogram form with Qiskit and Classiq. The example program we'll use is that which uses two qubits to create an entangled state (a Bell state).

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Create a Quantum Circuit with two qubits and two classical bits
qc = QuantumCircuit(2, 2)

# Apply Hadamard gate to the first qubit
qc.h(0)

# Apply CNOT gate (control is qubit 0, target is qubit 1)
qc.cx(0, 1)

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Visualize the circuit
qc.draw('mpl')
plt.show()

# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=1024)

# Grab results from the job
result = job.result()

# Get the counts (measurement results)
counts = result.get_counts(qc)
print("\nMeasurement results:", counts)

# Plot a histogram of the results
plot_histogram(counts)
plt.show()

##### 1.1.4 Decomposition

Decomposition is the method of scientifically breaking down a given quantum program into smaller computational operations or components. This is useful for investigation of quantum programs as well as for optmization techniques for specific quantum implementations. Let's now see how we can perform decomposition in Qiskit and how it's possible through Classiq in just a click of an interactive button! We will consider the Quantum Fourier Transform to understand hands-on how to perform decomposition.

In [ ]:
# Decomposition in Qiskit

from numpy import pi

def qft_rotations(circuit, n):
    if n == 0: # Exit function if circuit is empty
        return circuit
    n -= 1 # Indexes start from 0
    circuit.h(n) # Apply the H-gate to the most significant qubit
    for qubit in range(n):
        # For each less significant qubit, we need to do a
        # smaller-angled controlled rotation: 
        circuit.cp(pi/2**(n-qubit), qubit, n)

qc = QuantumCircuit(4)
qft_rotations(qc,4)
qc.draw()

decomposed_circuit = qc.decompose()
print("Decomposed QFT Circuit:")
decomposed_circuit.draw(output='mpl').show()

Add the video here for Classiq

##### 1.2 Similarities

One of the key similarities between Qiskit and Classiq lies in how we apply unitary operations or gates on qubits, although within the construct of Qiskit, gates are applied on qubits which are part of a circuit whereas in Classiq, it's a higher level function. Let's look at a simple example:

In [ ]:
# Qiskit 

qc = QuantumCircuit(1)
qc.h(0) # applying $H$ on the qubit
qc.y(0)

In [14]:
# Classiq

@qfunc
def operation(out: Output[QBit]):
    allocate(1, out) # initializing one qubit
    H(out)
    Y(out)

# To test this code, you may write a main function
    

<a id="section2"></a>2. Building Blocks of Quantum Algorithms

In this section of the tutorial, we will explore some building blocks of quantum computing, some of which will act as tools for building quantum algorithms discussed in the next section. We will also do some simple and fun exercises to get a hands-on understanding of how to program with Classsiq SDK!

##### 2.1 Preparing the initial state

Let's prepare the $|+\rangle$ state using Qiskit and see how the process is different with Classiq:

In [ ]:

from qiskit import QuantumCircuit

# Create a quantum circuit with 1 qubit
qc = QuantumCircuit(1)

# Initialize the qubit to the |+> state
qc.h(0)


What we do here in qiskit is to apply the Hadamard gate <code>(qc.h(0))</code> to the qubit which we know creates a superposition of $|0\rangle$ and $|1\rangle$ states, resulting in the $|+\rangle$ state. Now, let's see how to prepare this state using Classiq.

In [ ]:
from classiq import *

@qfunc
def prepare_plus_state(out: Output[QBit]):
    allocate(1, out)
    H(out)

This gives us an advantage of abstraction from using a lower level circuit entities as our building blocks and allows for repeated usage of this state preparation at various points in your quantum program.

##### 2.1.1 Exercise

Write a function that prepares the minus state $\ket{-}=\frac{1}{\sqrt2}(\ket{0}-\ket{1})$, assuming it recives the qubit $\ket{x}=\ket{0}$

<details>
<summary>
HINT
</summary>

Use `H(x)`,`X(x)`
</details>

Solution:

In [ ]:
@qfunc
def prepare_minus_state(x:QBit):
    X(x)
    H(x)

We can now test our code by calling the state preparation function without a main function:

In [ ]:
@qfunc
def main(x: Output[QBit]):
    allocate(1,x) # Initalize the qubit x
    prepare_minus_state(x) # Prepare the minus state

In [ ]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

In [ ]:
show(quantum_program)

Good job with the exercise! Let's continue.

##### 2.2 Constructing an Oracle

Often, quantum algorithms are built around investigating a given function $f(x)$ where we assume the function to be a black box that receives $x$ as input and gives $f(x)$ as its corresponding output. We will now explore how we can construct such oracles with Classiq in contrast to Qiskit.

**Detailed Example:** 

Let's assume that we want to build an oracle to determine whether a function $f(x)$ is balanced or constant. This is an interesting oracle which is the building block of the popular Deutsch-Jozsa Algorithm. Our predicate to check if the function is balanced or constant would be defined within a function <code>simple_predicate</code>.

Consider the predicate <code>res ^= x > 7</code>

This means that 
* whenever x > 7 is True, it evaluates 1 and res is XORed with 1, effectively toggling it to 0 or 1. 
* whenever x > 7 is False, res is XORed with 0, effectively always leaving its value unchanged

In simple words, this predicate is the question we want our oracle to answer. As you can clearly see in this example, the first scenario corresponds to the function being balanced and the latter to being constant. 

Observe keenly the <code>oracle</code> function in the below code snippet. We can see that the quantum circuit <code>qc</code> and the oracle itself are not inseparable, creating a dependency and lower-level design, which requires the oracle to be fed in with the quantum circuit, the function input, as well as an additional qubit for condition checking purposes.

In [ ]:
from qiskit import QuantumCircuit, Aer, execute

# Function oracle: f(x) = 1 if x > 7, otherwise f(x) = 0
def oracle(circuit, x, ancilla):
    for i in range(len(x)):
        circuit.cx(x[i], ancilla)
    # For x > 7, flip the ancilla qubit
    circuit.x(ancilla).c_if(x, 7)


In a stark constrast to the above code, Classiq simplifies the process by allowing the preparation of our initial state, independent declaration of the predicate that the oracle needs to check, and also leverages an in-built function <code>within_apply</code> to write mechanism of the oracle in a powerful way.

In [ ]:
# Oracle for Deutsch Jozsa Algorithm in Classiq

@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: constant_function(x, aux)
    )

In [ ]:
# Complete Working Example

from classiq import *

@qfunc
def constant_function(x: QNum, res: QBit):
    res ^= (x > 7)


@qfunc
def prepare_minus(out: Output[QBit]):
    allocate(1, out)
    X(out)
    H(out) 


@qfunc
def apply_oracle(x: QNum):
    aux = QBit("aux")
    within_apply(
        compute=lambda: prepare_minus(aux), action=lambda: constant_function(x, aux)
    )

@qfunc
def main(x: Output[QNum]):
    allocate(4, x)
    within_apply(compute=lambda: hadamard_transform(x), action=lambda: apply_oracle(x))

quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

##### 2.3 Changing Computational Basis

We will explore how we can work with compuational bases and change them for specific applications in this section of the tutorial with the example of Variational Quantum Eigensolver (VQE) algorithm. It's expert knowledge that VQE consists of rotational pauli and controlled-X operations. Qiskit enables us to use their transpilation tool to mention the ansatz circuit to use specified basis gates, in out example, <code>["rx", "ry", "rz", "cx"]</code>

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.primitives import Estimator
from qiskit.opflow import X, Z, I
from qiskit.algorithms.optimizers import COBYLA

# Define the Hamiltonian for a simple problem (e.g., H = X + Z)
hamiltonian = X + Z

# Define the ansatz (a simple two-local ansatz)
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', reps=1)

# Set up the VQE instance
vqe = VQE(ansatz, optimizer=COBYLA(), estimator=Estimator(), quantum_instance=Aer.get_backend('statevector_simulator'))

# Execute VQE to find the ground state energy
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)

# Get the resulting quantum circuit
circ = vqe.ansatz

# Transpile the circuit to the specified basis gates
transpiled_circ = transpile(circ, basis_gates=["rx", "ry", "rz", "cx"])

# Print the transpiled circuit
print(transpiled_circ)

Add video for Classiq

##### 2.4 Exporting Quantum Programs

QASM, or Quantum Assembly Language serves as one of the most intricate ways to export quantum programs from one SDK to other without hassle. Qiskit and Classiq both provide methods to export your quantum program using QASM but Classiq additionally offers other formats like LaTeX, HTML, JPEG, JSON, and transpiled QASM for other research purposes in addition to QASM enabling cross-SDk collaborations. Classiq provides an interactive button which enables users to swiftly export their programs.

Exporting a quantum circuit from Qiskit as QASM file

In [ ]:
from qiskit import QuantumCircuit
from qiskit.qasm2 import dumps # For exporting as a QASM string for further processing
from qiskit.qasm2 import dump # For exporting as a QASM file for cross-SDK usage
 
qc = QuantumCircuit(1)
 
qasm_str = dumps(qc)

with open("my_quantum_program.qasm", "w") as f:
    dump(qc, f)

Exporting a quantum program from Classiq as QASM file

Add video for Classiq

##### 2.5 Classiq's Unique Power: Optimization

In various specific quantum applications like Finance and Pharma, being able to optimize your quantum program by specifying certain constraints becomes inevitable. Some useful parameters used in Optimization of quantum programs are the width, depth, and gate count. It's not unknown that optimization using Qiskit would require deep domain knowledge in the art of advanced Optimization Techniques, which Classiq abstracts out for its users while still producing optimal results.

Example: Optimizing for Depth

In this example, we will demonstrate how we can optimize our quantum program for a maximum depth of 20 qubits:

Add the video here

<a id="section3"></a> 3. Quantum Algorithms: Phase Kickback

Phase Kickback is quantum primitive of high importance, used in larger quantum algorithms, e.g., Shor’s, Simon’s, Deutsch-Josza, and Grover’s. It deals with kicking the result of a function to the phase of a quantum state so it can be smartly manipulated with constructive and destructive interferences to achieve the desired result. This means that a phase shift applied to a control qubit during a controlled operation is "kicked back" to the control qubit.

3.1 Phase Kickback with Qiskit

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, execute
from qiskit.visualization import plot_histogram

# Define the number of qubits for the input
num_qubits = 4

# Create quantum and classical registers
x = QuantumRegister(num_qubits, 'x')
target = QuantumRegister(1, 'target')
c = ClassicalRegister(num_qubits, 'c')
qc = QuantumCircuit(x, target, c)

# Apply Hadamard transform to the x qubits
qc.h(x)

# Prepare the |-> state in the target qubit
qc.x(target)
qc.h(target)

# Oracle function: target ^= (x == 0)
# Apply X gates to flip x qubits for zero state detection
for i in range(num_qubits):
    qc.x(x[i])

# Apply a multi-controlled Toffoli gate (mct)
qc.mct(x[:], target)

# Apply X gates again to flip x qubits back
for i in range(num_qubits):
    qc.x(x[i])

# Measure the x qubits
qc.measure(x, c)

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend)
result = execute(transpiled_qc, backend, shots=1024).result()
counts = result.get_counts()

# Display the circuit and result
print(qc)
plot_histogram(counts).show()

On the higher level, we can observe from this code snippet that the logic depends heavily on applying the right gates and the core of the algorithm lies in flipping qubits based on the condition <code>target ^= (x == 0)</code> which we use an oracle to check for. Let's dig in deeper.

Firstly, we initialize x registers, which act as the target and control qubits manually. Then, the hadamard gate is applied to these x qubits indicating a hadamard transform. The target qubit is then utilized to prepare a $|-\rangle$ state. The purpose of the oracle here is to check if the state of each of these qubits is $|0\rangle$ using a fliping mechanism using $pauli X$ gate for the zero state detection. This is followed by the usage of the multi-controlled Toffoli gate which flips the target qubit only when all control qubits are in a specific state, in this case $|0\rangle$ after which the x qubits are flipped back before measurement.

3.2 Phase Kickback with Classiq

In [ ]:
from classiq import (
    H,
    Output,
    QBit,
    QNum,
    X,
    allocate,
    create_model,
    hadamard_transform,
    qfunc,
    show,
    synthesize,
    within_apply,
    write_qmod,
)


@qfunc
def prepare_minus(target: Output[QBit]):
    allocate(out=target, num_qubits=1)
    X(target)
    H(target)


@qfunc
def oracle_function(target: QBit, x: QNum):
    target ^= x == 0


@qfunc
def oracle_phase_kickback(x: QNum):
    target = QBit("target")
    within_apply(
        compute=lambda: prepare_minus(target), action=lambda: oracle_function(target, x)
    )


@qfunc
def main(x: Output[QNum]):
    allocate(num_qubits=4, out=x)
    hadamard_transform(x)
    oracle_phase_kickback(x)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

write_qmod(qmod, "phase_kickback")

Immediately upon viewing the above code snippet, we can realize stark differences from the equivalent code in qiskit. Let's explore them one by one.

The power of functional design allows us to build a function <code>prepare_minus_state</code> which encapsulates the operation of preparing a $|-\rangle$ using the pauli $X$ and hadamard $H$ gates, is the fact that it's reusable anywhere in your program later. It's nonteworthy that phase kickback is itself a primitive used within larger quantum programs, meaning resusablitily of functions can enable better software engineering practices and potentially, efficiency of your code.

<a id="section4"></a> 4. Appendix Notes

#### References

[1] Qiskit Textbook <a href="(https://github.com/Qiskit/textbook/tree/main/notebooks/ch-demos#)">(GitHub)</a>